#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader(r'E:\GenAI_Krish\Sec23_1-Basics+Of+Langchain\3.2-DataIngestion\speech.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs= text_splitter.split_documents(documents)

In [5]:
embeddings= (
    OllamaEmbeddings(model="mxbai-embed-large")
)
db=FAISS.from_documents(docs, embeddings)
db

In [7]:
### querying

query= "What does the speaker believe is the main reason the united states should senter the war?"
query1= "How does the speaker describe the desired outcome of the war?"

In [9]:
docs= db.similarity_search(query)
docs[0]

Document(id='adb0a3db-60e2-438b-9c70-ae6248e67686', metadata={'source': 'E:\\GenAI_Krish\\Sec23_1-Basics+Of+Langchain\\3.2-DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.')

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [11]:
retriver= db.as_retriever()
docs=retriver.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [12]:
docs_Score= db.similarity_search_with_score(query)
docs_Score

[(Document(id='adb0a3db-60e2-438b-9c70-ae6248e67686', metadata={'source': 'E:\\GenAI_Krish\\Sec23_1-Basics+Of+Langchain\\3.2-DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(211.4506)),
 (Document(id=

In [14]:
embedding_vectors_query = embeddings.embed_query(query)
embedding_vectors_query

[0.5237599611282349,
 0.1687507927417755,
 0.32029446959495544,
 0.9985927939414978,
 -0.3490802049636841,
 -0.34629133343696594,
 -0.20320631563663483,
 0.26301494240760803,
 -0.42399829626083374,
 0.11803047358989716,
 -0.2811742424964905,
 -0.30210909247398376,
 -0.24191230535507202,
 -0.05895476043224335,
 -0.00021346285939216614,
 -0.5087652206420898,
 -0.7781909108161926,
 0.40623635053634644,
 0.6731712818145752,
 0.09606346487998962,
 -0.3705191910266876,
 0.0718885064125061,
 -0.23338055610656738,
 -0.42662376165390015,
 -0.32713639736175537,
 -0.1430075466632843,
 -0.2309696525335312,
 0.31354057788848877,
 0.6424189805984497,
 0.7271415591239929,
 0.06755473464727402,
 -0.3082260489463806,
 -0.34758850932121277,
 -1.2805969715118408,
 0.3174460828304291,
 -0.4875779151916504,
 0.46667706966400146,
 -1.6663837432861328,
 -0.2760273814201355,
 -0.1999637931585312,
 -0.13029319047927856,
 0.02517392486333847,
 0.7488057017326355,
 -0.19694800674915314,
 -0.5577818155288696,
 -0

In [15]:
docs_Score1=db.similarity_search_by_vector(embedding_vectors_query)
docs_Score1

[Document(id='adb0a3db-60e2-438b-9c70-ae6248e67686', metadata={'source': 'E:\\GenAI_Krish\\Sec23_1-Basics+Of+Langchain\\3.2-DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='12b207a2-2538-48c0-a60a-8d

In [16]:
### Vector DB Saving
db.save_local("fiass_db")

In [19]:
### Loading the Stored Vector DB
New_db= FAISS.load_local("fiass_db",embeddings,allow_dangerous_deserialization=True)
docs=New_db.similarity_search(query)

#### Chroma
Chroma is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

https://python.langchain.com/v0.2/docs/integrations/vectorstores/

In [20]:
## building a sample vectorDB
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [21]:
loader = TextLoader(r'E:\GenAI_Krish\Sec23_1-Basics+Of+Langchain\3.2-DataIngestion\speech.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits= text_splitter.split_documents(documents)

In [22]:
embeddings= (
    OllamaEmbeddings(model="mxbai-embed-large")
)
vectordb=Chroma.from_documents(docs, embeddings)
vectordb

In [24]:
### query it
query= "What does the speaker believe is the main reason the united states should senter the war?"
docs = vectordb.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [25]:
## SAVE LOCAL
vectordb=Chroma.from_documents(documents=splits,embedding=embeddings,persist_directory="./chroma_db")


In [26]:
## load from Chroma disk
db2= Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
docs=db2.similarity_search(query)
print(docs[0].page_content)

It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,


In [27]:
##Similarity Search With Score
docs = vectordb.similarity_search_with_score(query)
docs

[(Document(id='efbec7cd-a95b-4093-84c8-d04b7acba8b6', metadata={'source': 'E:\\GenAI_Krish\\Sec23_1-Basics+Of+Langchain\\3.2-DataIngestion\\speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'),
  220.30951277674842),
 (Document(id='713cdd03-a41a-4bec-a2d1-70bfbe2ed03b', metadata={'source': 'E:\\GenAI_Krish\\Sec23_1-Basics+Of+Langchain\\3.2-DataIngestion\\speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who 

In [29]:
### Retriver Option
retriver= vectordb.as_retriever()
retriver.invoke(query)[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'